# 5. Catalyst + Oscar

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.add("Symbolics")

In [219]:
using Symbolics
using Catalyst
using Catalyst: unknowns # sometimes need to specify which method to use if defined in multiple packages
using Oscar
using Oscar: groebner_basis

### Aside: Symbolics.jl
Catalyst uses `Symbolics.jl` as its computer algebra system. This is slightly different to Oscar.

In [226]:
@variables x y z

3-element Vector{Num}:
 x
 y
 z

In [227]:
x isa Number  # true

f(t) = t^2 + t
sym_vec = f.([2x, y^2 + 1, x + z + y])

3-element Vector{Num}:
         2x + (4//1)*(x^2)
     1 + y^2 + (1 + y^2)^2
 x + y + z + (x + y + z)^2

In [228]:
Symbolics.expand.(sym_vec)

3-element Vector{Num}:
                          (2//1)*x + (4//1)*(x^2)
                                 2 + 3(y^2) + y^4
 x + y + z + x^2 + 2x*y + 2x*z + y^2 + 2y*z + z^2

## Catalyst to Oscar

In [229]:
rn = @reaction_network begin
    k, 2A + 3B --> A + 2C + D
    b, C + D --> 2A + 3B
end

Model ##ReactionSystem#354:
Unknowns (4): see unknowns(##ReactionSystem#354)
  A(t)
  B(t)
  C(t)
  D(t)
Parameters (2): see parameters(##ReactionSystem#354)
  k
  b

In [230]:
odesys = convert(ODESystem, rn, combinatoric_ratelaws=false)

Model ##ReactionSystem#354:
Equations (4):
  4 standard: see equations(##ReactionSystem#354)
Unknowns (4): see unknowns(##ReactionSystem#354)
  A(t)
  B(t)
  C(t)
  D(t)
Parameters (2): see parameters(##ReactionSystem#354)
  k
  b

We can extract the mass-action polynomials from the ODESystem struct.

In [231]:
@show vars = unknowns(odesys)
@show params = parameters(odesys)
eqn_rhs = [eq.rhs for eq in equations(odesys)]

vars = unknowns(odesys) = SymbolicUtils.BasicSymbolic{Real}[A(t), B(t), C(t), D(t)]
params = parameters(odesys) = SymbolicUtils.BasicSymbolic{Real}[k, b]


4-element Vector{SymbolicUtils.BasicSymbolic{Real}}:
 2b*D(t)*C(t) - k*(B(t)^3)*(A(t)^2)
 3b*D(t)*C(t) - 3k*(B(t)^3)*(A(t)^2)
 -b*D(t)*C(t) + 2k*(B(t)^3)*(A(t)^2)
 -b*D(t)*C(t) + k*(B(t)^3)*(A(t)^2)

We first need to convert between the symbolic representations of Catalyst and Oscar (this is the most fiddly part).

In [232]:
function polynomial_ideal(eqn_rhs, vars, params)
    # Convert Catalyst symbolic variables into Julia Symbol types
    varnames = tosymbol.(vars, escape=false)
    paramnames = tosymbol.(params)
    
    # Create polynomial ring in Oscar
    CC, oscar_coeffs = polynomial_ring(QQ, paramnames)
    ff = fraction_field(CC)
    RR, oscar_vars = polynomial_ring(ff, varnames)
    
    # Map Catalyst variables to Oscar variables
    cat_var_params = [vars; params]
    oscar_var_params = [oscar_vars; oscar_coeffs]
    cat_to_oscar = Dict(cat => oscar for (cat, oscar) in zip(cat_var_params, oscar_var_params))
    oscar_to_cat = Dict((oscar => cat) for (cat, oscar) in cat_to_oscar)
    
    # build Oscar polynomial by substituting oscar vars in Catalyst equations RHS (Right Hand Sides)
    polys = map(eqn_rhs) do rhs
        if rhs isa Number  # is a constant e.g. zero
            RR(rhs)
        else
            Symbolics.substitute(rhs, cat_to_oscar)
        end
    end
    ideal(RR, polys)
end

I = polynomial_ideal(eqn_rhs, vars, params)

Ideal generated by
  -k*A^2*B^3 + 2*b*C*D
  -3*k*A^2*B^3 + 3*b*C*D
  2*k*A^2*B^3 - b*C*D
  k*A^2*B^3 - b*C*D

Now we can find a Groebner basis for $I$

In [233]:
G = groebner_basis(I)

Gröbner basis with elements
  1: C*D
  2: k*A^2*B^3 - b*C*D
with respect to the ordering
  degrevlex([A, B, C, D])

In [234]:
# we may want to map back to Catalyst/Symbolics
function to_symbolic_polynomial(poly::MPolyRingElem)
    # create new symbolic vars from ring of poly
    var_syms = Symbolics.variable.(gens(parent(poly)))
    coeff_syms = Symbolics.variable.(gens(coefficient_ring(poly)))

    # coefficients
    coeff_terms = Oscar.coefficients(poly)
    coeffs = zeros(Num, length(coeff_terms))
    for (i, term) in enumerate(coeff_terms)
        cf_and_es = coefficients_and_exponents(term.num)
        coeff_polyn = sum(Int(cf) * prod(coeff_syms .^ es) for (cf, es) in cf_and_es)
        coeffs[i] = coeff_polyn
    end
    # polynomial variables
    exp_vecs = collect(Oscar.exponents(poly))
    xs = [prod(var_syms .^ e_vec) for e_vec in exp_vecs]
    coeffs' * xs
end

to_symbolic_polynomial.(gens(G))

2-element Vector{Num}:
                    C*D
 -C*D*b + (A^2)*(B^3)*k

## Example: Dimer production

In [235]:
dimer_production = @reaction_network begin
    λₘ, 0 --> mRNA
    λₚ, mRNA --> mRNA + P
    (k₁, k₂), 2P <--> P₂
    δ, (mRNA, P, P₂) --> 0
end

Model ##ReactionSystem#360:
Unknowns (3): see unknowns(##ReactionSystem#360)
  mRNA(t)
  P(t)
  P₂(t)
Parameters (5): see parameters(##ReactionSystem#360)
  λₘ
  λₚ
  k₁
  k₂
  ⋮

In [236]:
odesys = convert(ODESystem, dimer_production, combinatoric_ratelaws=true)

Model ##ReactionSystem#360:
Equations (3):
  3 standard: see equations(##ReactionSystem#360)
Unknowns (3): see unknowns(##ReactionSystem#360)
  mRNA(t)
  P(t)
  P₂(t)
Parameters (5): see parameters(##ReactionSystem#360)
  λₘ
  λₚ
  k₁
  k₂
  ⋮

In [237]:
function polynomial_ideal(odesys::ODESystem)
    vars = unknowns(odesys)
    params = parameters(odesys)
    eqn_rhs = [eq.rhs for eq in equations(odesys)]
    polynomial_ideal(eqn_rhs, vars, params)
end
I = polynomial_ideal(odesys)

Ideal generated by
  -δ*mRNA + λₘ
  λₚ*mRNA - k₁*P^2 - δ*P + 2*k₂*P₂
  1//2*k₁*P^2 + (-k₂ - δ)*P₂

In [238]:
G = groebner_basis(I)

Gröbner basis with elements
  1: δ^2*P + 2*δ^2*P₂ - λₘ*λₚ
  2: δ*mRNA - λₘ
  3: 4*k₁*δ^2*P₂^2 + λₘ*λₚ*k₁*P + (-2*λₘ*λₚ*k₁ - 2*k₂*δ^2 - 2*δ^3)*P₂
with respect to the ordering
  degrevlex([mRNA, P, P₂])

In [239]:
@assert dim(I) == 0
G = groebner_basis(I, algorithm=:fglm, ordering=lex(base_ring(I)))

Gröbner basis with elements
  1: 4*k₁*δ^4*P₂^2 + (-4*λₘ*λₚ*k₁*δ^2 - 2*k₂*δ^4 - 2*δ^5)*P₂ + λₘ^2*λₚ^2*k₁
  2: δ^2*P + 2*δ^2*P₂ - λₘ*λₚ
  3: δ*mRNA - λₘ
with respect to the ordering
  lex([mRNA, P, P₂])

In [240]:
Oscar.vars.(gens(G))

3-element Vector{Vector{AbstractAlgebra.Generic.MPoly{AbstractAlgebra.Generic.FracFieldElem{QQMPolyRingElem}}}}:
 [P₂]
 [P, P₂]
 [mRNA]

In [241]:
to_symbolic_polynomial.(gens(G))

3-element Vector{Num}:
 k₁*(λₘ^2)*(λₚ^2) + P₂*(-4k₁*(δ^2)*λₘ*λₚ - 2k₂*(δ^4) - 2(δ^5)) + 4(P₂^2)*k₁*(δ^4)
                                                     -λₘ*λₚ + P*(δ^2) + 2P₂*(δ^2)
                                                                     -λₘ + mRNA*δ

## Example: Cholesterol Regulation RPA
*Robust homeostasis of cellular cholesterol is a consequence of endogenous antithetic integral control*
([Scheepers and Araujo, 2023](https://www.frontiersin.org/journals/cell-and-developmental-biology/articles/10.3389/fcell.2023.1244297/full))

In [242]:
rn = include("./data/cholesterol_rn.jl")

Model Cholesterol_RN:
Unknowns (14): see unknowns(Cholesterol_RN)
  S_p(t)
  C(t)
  P(t)
  R(t)
  ⋮
Parameters (23): see parameters(Cholesterol_RN)
  k_1
  k_2
  k_3
  k_4
  ⋮

In [243]:
odesys = convert(ODESystem, rn)
I = polynomial_ideal(odesys)

Ideal generated by
  -k_13*S_p + k_1*S_ci
  -η*C*S_ci + θ*C_e
  p_3*S_p - k_16*P
  -k_14*P*R - k_11*R + p_1*S_r
  -k_2*S_r + k_1*S_ci
  -k_10*S_h + k_1*S_ci
  -k_9*H*H_R + α
  p_2*S_h - k_15*H_R*C_e
  -k_5*C_p + k_4*C_f + k_6*C_e
  -k_8*E + k_7*C_e
  -η*C*S_ci + μ
  k_3*R*C_L - k_4*C_f
  k_9*H*H_R + k_5*C_p + k_8*E + (-k_6 - k_7 - k_12)*C_e
  0

In [244]:
# eliminate all vars EXCEPT C_f, C_L
xs = gens(base_ring(I))
elimination_ordering = degrevlex(xs[[1:11;13]]) * degrevlex(xs[[12, 14]])
G = Oscar.groebner_basis(I; ordering=elimination_ordering)

Gröbner basis with elements
  1: k_4*θ*C_f - k_12*μ + θ*α
  2: θ*C_e - μ
  3: k_1*k_3*k_13*k_16*p_1*θ*S_ci*C_L + (-k_1*k_2*k_12*k_14*p_3*μ + k_1*k_2*k_14*p_3*θ*α)*S_ci - k_2*k_4*k_11*k_13*k_16*θ*C_f
  4: k_8*E - k_7*C_e
  5: k_5*C_p - k_6*C_e - k_4*C_f
  6: k_10*k_15*μ*H_R - k_1*p_2*θ*S_ci
  7: (k_2*k_9*k_11*k_12*k_13*k_16*p_2*μ*θ - k_2*k_9*k_11*k_13*k_16*p_2*θ^2*α)*H + k_3*k_5*k_10*k_13*k_15*k_16*p_1*μ*θ*C_p*C_L + (-k_2*k_5*k_10*k_12*k_14*k_15*p_3*μ^2 + k_2*k_5*k_10*k_14*k_15*p_3*μ*θ*α)*C_p + k_3*k_8*k_10*k_13*k_15*k_16*p_1*μ*θ*E*C_L + (-k_2*k_8*k_10*k_12*k_14*k_15*p_3*μ^2 + k_2*k_8*k_10*k_14*k_15*p_3*μ*θ*α)*E + (-k_3*k_6*k_10*k_13*k_15*k_16*p_1*μ*θ - k_3*k_7*k_10*k_13*k_15*k_16*p_1*μ*θ - k_3*k_10*k_12*k_13*k_15*k_16*p_1*μ*θ)*C_e*C_L + (k_2*k_6*k_10*k_12*k_14*k_15*p_3*μ^2 - k_2*k_6*k_10*k_14*k_15*p_3*μ*θ*α + k_2*k_7*k_10*k_12*k_14*k_15*p_3*μ^2 - k_2*k_7*k_10*k_14*k_15*p_3*μ*θ*α + k_2*k_10*k_12^2*k_14*k_15*p_3*μ^2 - k_2*k_10*k_12*k_14*k_15*p_3*μ*θ*α)*C_e
  8: k_10*S_h - k_1*S_ci
  9: k

In [245]:
G[1]

k_4*θ*C_f - k_12*μ + θ*α

In [246]:
function lift_symbolics(f, I, xs)
    A = coordinates(f, I)
    B = zeros(Num, length(A))
    for (i, aa) in enumerate(A)
        if length(aa.coeffs) == 0
            continue
        else
            ff_coeff = aa.coeffs[]
            cf_and_es = coefficients_and_exponents(ff_coeff.num)
            polyn = sum(Int(cf) * prod(xs .^ es) for (cf, es) in cf_and_es)
            B[i] = polyn
        end
    end
    return B
end

params = parameters(rn)
sym_coeffs = lift_symbolics(G[1], I, params)

14-element Vector{Num}:
     0
  k_12
     0
     0
     0
     0
     θ
     0
     θ
     θ
 -k_12
     0
     θ
     0

In [247]:
eqn_lhs = [eqn.lhs for eqn in equations(odesys)]
eqn_rhs = [eqn.rhs for eqn in equations(odesys)]

dot(eqn_lhs, sym_coeffs) |> Symbolics.expand |> display
rho = dot(eqn_rhs, sym_coeffs) |> Symbolics.expand

k_12*Differential(t)(C(t)) - k_12*Differential(t)(S_ci(t)) + Differential(t)(C_p(t))*θ + Differential(t)(E(t))*θ + Differential(t)(C_e(t))*θ + Differential(t)(H(t))*θ

-k_12*μ + α*θ + k_4*C_f(t)*θ

In [248]:
setpoint = Symbolics.symbolic_solve(rho, rn.C_f)[]
rn.C_f ~ setpoint

C_f(t) ~ (-k_12*μ + α*θ) / (-k_4*θ)